In [16]:
import math

In [36]:
class Phase2:
    """
    Расчитывает параметры тренировок по 2 фазе плана Джека Дениелса
    'От 800 метров до марафона' второе издание
    """
    def __init__(self, week_distance):
        self.week_distance = week_distance # недельный километраж
        self.easy_week_distance = week_distance * 0.25
        self.repeats_week_distance = week_distance * 0.05
        
    def calculate_distance(self, time_long: int, pace_long: str) -> float:
        """
        Расчитывает дистанцию длительной тренировки 25% от
        недельного километража или дистанцмю по заплонированному времени.
        Выводит меньшую дистанцию
        
        Параметры:
        time_long: Время длительного забега по плану в минутах
        
        Возвращаемое значение:
        float: Дистанцию на длительную тренировку
        """
        # Разделяем темп на минуты и секунды
        pace_minutes, pace_seconds = map(int, pace_long.split(':'))

        # Переводим темп в секунды
        pace_total_seconds = pace_minutes * 60 + pace_seconds

        # Переводим общее время в секунды
        total_time_seconds = time_long * 60

        # Вычисляем расстояние в километрах
        distance_km = total_time_seconds / pace_total_seconds
        result_distance = round(min(distance_km, self.easy_week_distance), 2)

        return result_distance
    
    def number_episodes(self, plan_repeats: list, number_training: int) -> int:
        """
        Метод расчитывает количество серий повторов в Пв-темпе
        
        Параметры:
        plan_repeats (list): Список повторов в тренировоке. Например [(200, 200), (200, 400)]
        number_training (int): Номер К тренировки.

        Возвращаемое значение:
        int: Количество серий повторов в Пв-темпе.
        
        """
        wd = self.repeats_week_distance
        if number_training == 1:
            wd = wd * .75 # оставить 25% дистанции для второй тренировки
            
        total_repeats = sum(pair[0] for pair in plan_repeats) / 1000
        nm = wd / total_repeats
        nm = math.floor(nm) # округлить до целого в меньшую сторону
        if number_training == 1:
            self.repeats_week_distance = self.repeats_week_distance - total_repeats * nm
        
        return nm
    


In [40]:
p2 = Phase2(35) 
print(p2.week_distance)
print(p2.easy_week_distance)
print(f"дистанция повторов {p2.repeats_week_distance}")
dist = p2.calculate_distance(120, '7:00')
print(dist)
num_1 = p2.number_episodes([(200, 200), (200, 400), (400, 200)],1)
num_2 = p2.number_episodes([(200, 200)],2)
print(f"в первой тренировке количество серий - {num_1}\nво второй - {num_2} ")

35
8.75
дистанция повторов 1.75
8.75
в первой тренировке количество серий - 1
во второй - 4 
